<a href="https://colab.research.google.com/github/0xVolt/whats-up-doc/blob/main/src/experimental-notebooks/code_trans_t5_small_code_documentation_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install -q --no-cache transformers datasets pytorch_lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 304.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.6/774.6 kB 222.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 190.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 189.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 206.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 186.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 231.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 318.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 174.8 MB/s eta 0:00:00


In [3]:
import transformers
from transformers import AutoTokenizer, AutoModelWithLMHead
from datasets import load_dataset
from pytorch_lightning import LightningDataModule, LightningModule, Trainer
from torch.optim import AdamW
from torch.utils.data import DataLoader

In [4]:
class CodeTransDataModule(LightningDataModule):
    def __init__(self, dataset, tokenizer, batch_size=16):
        super().__init__()
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.batch_size = batch_size
        # Change the instruction column to description and the output column to code
        self.dataset = self.dataset.rename_columns({"instruction": "description", "output": "code"})

    def train_dataloader(self):
        return DataLoader(
            self.dataset,
            batch_size=self.batch_size,
            shuffle=True,
            collate_fn=self.collate_fn,
        )

    def val_dataloader(self):
        return DataLoader(
            self.dataset,
            batch_size=self.batch_size,
            shuffle=False,
            collate_fn=self.collate_fn,
        )

    def test_dataloader(self):
        return DataLoader(
            self.dataset,
            batch_size=self.batch_size,
            shuffle=False,
            collate_fn=self.collate_fn,
        )

    def collate_fn(self, batch):
        input_ids = self.tokenizer([x["code"] for x in batch], padding=True, truncation=True, return_tensors="pt").input_ids
        labels = self.tokenizer([x["description"] for x in batch], padding=True, truncation=True, return_tensors="pt").input_ids

        # Cast the mask tensor to a floating-point tensor before adding it to the position bias tensor
        mask = self.tokenizer([x["code"] for x in batch], return_tensors="pt").attention_mask.float()
        position_bias = position_bias + mask  # (batch_size, n_heads, seq_length, key_length)

        return {"input_ids": input_ids, "labels": labels, "position_bias": position_bias}

        # input_ids = self.tokenizer([x["instruction"] + " " + x["prompt"] for x in batch], padding=True, truncation=True, return_tensors="pt").input_ids
        # labels = self.tokenizer([x["code"] for x in batch], return_tensors="pt").input_ids
        # return {"input_ids": input_ids, "labels": labels}

In [5]:
class CodeTransLightningModule(LightningModule):
    def __init__(self, model, learning_rate=1e-3):
        super().__init__()
        self.model = model
        self.learning_rate = learning_rate

    def forward(self, input_ids, labels):
        outputs = self.model(input_ids=input_ids, labels=labels)
        return outputs.loss

    def training_step(self, batch, batch_idx):
        loss = self.model(batch["input_ids"], batch["labels"])
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        loss = self.model(batch["input_ids"], batch["labels"])
        self.log("val_loss", loss)

    def test_step(self, batch, batch_idx):
        loss = self.model(batch["input_ids"], batch["labels"])
        self.log("test_loss", loss)

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=self.learning_rate)
        return optimizer

In [ ]:
# Load the pre-trained code trans model
model = AutoModelWithLMHead.from_pretrained("SEBIS/code_trans_t5_base_source_code_summarization_python_transfer_learning_finetune")

# Load the dataset
dataset = load_dataset("iamtarun/python_code_instructions_18k_alpaca")
dataset = dataset['train']

# Create the tokenizer
tokenizer = AutoTokenizer.from_pretrained("SEBIS/code_trans_t5_base_source_code_summarization_python_transfer_learning_finetune")

# Create the LightningDataModule
data_module = CodeTransDataModule(dataset, tokenizer)

# Create the Trainer
trainer = Trainer()

# Train the model
trainer.fit(CodeTransLightningModule(model), data_module)

# Evaluate the model
trainer.test(CodeTransLightningModule(model), data_module)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1499: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
